In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


In [2]:
import pandas as pd
import urllib.request
import json
import datetime

In [3]:
for i in range(1,8):
  today = datetime.datetime.today()
  lastday = (today - datetime.timedelta(i)).strftime('%Y%m%d') 
  url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

  response = urllib.request.urlopen(url+lastday).read()
  response = json.loads(response)

  data = pd.DataFrame(response['data'])
  try:
    print(i)
    print(str(data['SALEDATE'][0]))
    data.to_csv("/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/"+str(data['SALEDATE'][0])+".csv")
  except:
    print(i)
    df = pd.DataFrame.from_dict([{'PUM_NM' : 0,
                              'LV_NM' : 0,
                              'TOT_AMT' : 0,
                              'SAN_NM' : 0,
                              'SALEDATE' : str(lastday),
                              'CMP_NM' : 0,
                              'DAN_NM' : 0,
                              'WHSAL_NM' : 0,
                              'SIZE_NM' : 0,
                              'COST' : 0,
                              'POJ_NM' : 0,
                              'TOT_QTY' : 0,
                              'QTY' : 0,
                              'KIND_NM' : 0,
                              'DANQ' : 0}])
    df.to_csv("/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/"+str(df['SALEDATE'][0])+".csv")
    continue

1
20211104
2
20211103
3
20211102
4
20211101
5
5
6
20211030
7
20211029


In [4]:
import pandas as pd
import numpy as np
from glob import glob
import multiprocessing
import pickle
import csv

In [5]:
# 윈도우즈 사용시 함수를 별도의 .py파일로 저장 후 import하여 사용
def preprocessing(tsalet_file):
    unique_pum = [
        '배추', '무', '양파', '건고추','마늘',
        '대파', '얼갈이배추', '양배추', '깻잎',
        '시금치', '미나리', '당근',
        '파프리카', '새송이', '팽이버섯', '토마토',
    ]
    
    unique_kind = [
        '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
    ]
    
    train_dict = {
        'date':0
    }
    
    for sub in unique_pum:
        train_dict[f'{sub}_거래량(kg)'] = 0
        train_dict[f'{sub}_가격(원/kg)'] = 0
        
    for sub in unique_kind:
        train_dict[f'{sub}_거래량(kg)'] = 0
        train_dict[f'{sub}_가격(원/kg)'] = 0
        
    tsalet_sample = pd.read_csv(tsalet_file)
    days = sorted(tsalet_sample['SALEDATE'].unique())
    for day in days:
        train_dict.update(date=day)
        for sub in unique_pum:
            # 날짜별, 품목별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['PUM_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]
            if c.shape[0] == 0:
                train_dict[f'{sub}_거래량(kg)']=0
                train_dict[f'{sub}_가격(원/kg)']=0
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = tot_amt/(tot_qty+1e-20)
                train_dict[f'{sub}_거래량(kg)']=tot_qty
                train_dict[f'{sub}_가격(원/kg)']=mean_price
        
        for sub in unique_kind:
            # 날짜별, 품종별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['KIND_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]
            if c.shape[0] == 0:
                train_dict[f'{sub}_거래량(kg)']=0
                train_dict[f'{sub}_가격(원/kg)']=0
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = round(tot_amt/(tot_qty+1e-20))
                tot_qty = round(tot_qty, 1)
                train_dict[f'{sub}_거래량(kg)']=tot_qty
                train_dict[f'{sub}_가격(원/kg)']=mean_price
                
    return(train_dict)

In [6]:
tsalet_files = sorted(glob('/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/*.csv'))
tsalet_files

['/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/20210101.csv',
 '/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/20210102.csv',
 '/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/20210103.csv',
 '/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/20210104.csv',
 '/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/20210105.csv',
 '/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/20210106.csv',
 '/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/20210107.csv',
 '/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/20210108.csv',
 '/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/test/20210109.csv',
 '/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/private_data/

In [7]:
with open('/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/test.csv', 'w', newline='') as csvfile:
  fieldnames = preprocessing(tsalet_files[0])
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  writer.writeheader()
  writer.writerow(preprocessing(tsalet_files[0]))

In [8]:
for i in range(1, len(tsalet_files)):
  with open('/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/test.csv', 'a', newline='') as csvfile:
    fieldnames = preprocessing(tsalet_files[i])
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow(preprocessing(tsalet_files[i]))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or s